In [1]:
import pandas as pd
import numpy as np
import string
from matplotlib import pyplot as plt

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import recall_score, precision_recall_curve, confusion_matrix
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import Pipeline

import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import SnowballStemmer
nltk.download('punkt')

from sklearn.naive_bayes import MultinomialNB

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\vlads\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [2]:
# Загрузка данных
data = pd.read_csv('data/482reviews.csv')

In [3]:
data.shape

(482, 2)

In [4]:
data.head(5)

,review,assessment
0,"+красивая, качественная печать, качественые де...",1
1,3 Д ручки очень интересное увлечение для детей...,0
2,"4 года - буквы вообще учить не хочет, ничем ин...",1
3,"Азул — это та самая игрулька, куда мозги можно...",1
4,"Бочонки маленькие, фишки обычный картон. Не по...",0


In [5]:
data['assessment'].value_counts()

assessment
1    268
0    214
Name: count, dtype: int64

In [6]:
X_train, X_test, y_train, y_test = train_test_split(data['review'], data['assessment'], test_size = 0.25, random_state = 1)

In [7]:
y_train.value_counts()

assessment
1    191
0    170
Name: count, dtype: int64

In [8]:
y_test.value_counts()

assessment
1    77
0    44
Name: count, dtype: int64

In [9]:
sentence_example = data.iloc[1]['review']
sentence_example

'3 Д ручки очень интересное увлечение для детей и взрослых. Покупали дочке в подарок на 8 лет, очень хотела она. А тут разочарование... Выполнив все первоначальные действия по инструкции, загрузили пластик для изготовления рисунка и в  момент подачи носик самой ручки стал плавиться. Дочь очень расстроилась! (((( Возврат оформили. Хотим всё же перезаказать данную модель, может то был единичный брак...., '

In [10]:
#Предобработка текста
sentence_example = data.iloc[1]['review']

snowball = SnowballStemmer(language = "russian")
russian_stop_words = stopwords.words("russian")

def tokenize_sentence(sentence: str, remove_stop_words: bool = True):
    tokens = word_tokenize(sentence, language = "russian")
    tokens = [i for i in tokens if i not in string.punctuation]
    if remove_stop_words:
        tokens = [i for i in tokens if i not in russian_stop_words]
    tokens = [snowball.stem(i) for i in tokens]
    return tokens

In [11]:
tokenize_sentence(sentence_example)

['3',
 'д',
 'ручк',
 'очен',
 'интересн',
 'увлечен',
 'дет',
 'взросл',
 'покупа',
 'дочк',
 'подарок',
 '8',
 'лет',
 'очен',
 'хотел',
 'а',
 'разочарован',
 '...',
 'выполн',
 'первоначальн',
 'действ',
 'инструкц',
 'загруз',
 'пластик',
 'изготовлен',
 'рисунк',
 'момент',
 'подач',
 'носик',
 'сам',
 'ручк',
 'стал',
 'плав',
 'доч',
 'очен',
 'расстро',
 'возврат',
 'оформ',
 'хот',
 'все',
 'перезаказа',
 'дан',
 'модел',
 'единичн',
 'брак',
 '....']

In [12]:
vectorizer = TfidfVectorizer(tokenizer = lambda x: tokenize_sentence(x,  remove_stop_words = True), token_pattern=None)

In [13]:
features = vectorizer.fit_transform(X_train)

In [14]:
model = LogisticRegression(random_state = 0)
model.fit(features, y_train)

LogisticRegression(random_state=0)

In [15]:
X = vectorizer.fit_transform(X_train)
y_pred = model.predict(X)

In [16]:
model.predict(features[40])

array([0], dtype=int64)

In [17]:
X_train.iloc[40]

'Ужасное качество исполнения. Отвратительное. Нельзя использовать здесь ненормативную лексику, а ой как хочется. Скобы степлера, которыми крепятся "сетки" в лузах частично пристреляны мимо. Борта, сделанные из ДСП, местами в сколах, хотя это не влияет на игру, конечно. Сукно на бортах плохо натянуто, потому ждать какого-то хорошего отскока шаров от них не придётся. Посреди самого стола, в самом центре, огромное пятно почти с половину стола, очень хотелось бы думать, что это клей, которых сукно приклеили к фанере стола. Короче, поставил бы одну звезду, но подарок ребёнку всё равно нравится, играют со старшим братом. Дороже 500 рублей бы не купил. Да вообще бы не купил, если бы жена проверила на качтство при получении., '

In [18]:
model_pipeline = Pipeline([
    ("vectorizer", TfidfVectorizer(tokenizer = lambda x: tokenize_sentence(x, remove_stop_words=True), token_pattern=None)),
    ("model", LogisticRegression(random_state = 0 ))])

In [19]:
model_pipeline.fit(X_train, y_train)

Pipeline(steps=[('vectorizer',
                 TfidfVectorizer(token_pattern=None,
                                 tokenizer=<function <lambda> at 0x000001D21D9C3D80>)),
                ('model', LogisticRegression(random_state=0))])

In [20]:
model_pipeline.predict(['Бункер - вообще не стоит своих денег! Мы в советскую монополию играем снова и снова, а в эту на один раз.'])

array([0], dtype=int64)

In [21]:
model_pipeline.predict(['Классная, заинтересовала 2-ух летку и 4-ех , периодически достаем, играем),'])

array([1], dtype=int64)

In [22]:
# Прогнозирование
y_pred = model_pipeline.predict(X_test)

In [23]:
pd.DataFrame(
    confusion_matrix(y_test, y_pred),
    index=[["actual", "actual"], ["negative", "positive"]],
    columns=[["predicted", "predicted"], ["negative", "positive"]],
)

predicted         
                 negative positive
actual negative        35        9
       positive         5       72

In [ ]:
# Оценка качества модели
# Добавил импортов
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report

accuracy = accuracy_score(y_test, y_pred)
report = classification_report(y_test, y_pred)

print(f'Accuracy: {accuracy}')
print('Classification Report:')
print(report)

Accuracy: 0.8842975206611571
Classification Report:
              precision    recall  f1-score   support

           0       0.88      0.80      0.83        44
           1       0.89      0.94      0.91        77

    accuracy                           0.88       121
   macro avg       0.88      0.87      0.87       121
weighted avg       0.88      0.88      0.88       121



Если мы хотим найти 95% негативных комментариев, то...

In [ ]:
from sklearn.metrics import precision_score
# Добавил импорт

precision_score(y_test, y_pred)

0.8888888888888888

In [27]:
recall_score(y_test, y_pred)

0.935064935064935

In [28]:
prec, rec, thresholds = precision_recall_curve(y_test, probas_pred=model_pipeline.predict_proba(X_test)[:, 1])

c:\Users\vlads\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\metrics\_ranking.py:993: FutureWarning: probas_pred was deprecated in version 1.5 and will be removed in 1.7.Please use ``y_score`` instead.
  warnings.warn(


In [29]:
np.where(prec > 0.95)

(array([ 53,  54,  55,  56,  61,  62,  63,  64,  65,  66,  67,  68,  69,
         70,  71,  72,  73,  74,  75,  76,  77,  78,  79,  80,  81,  82,
         83,  84,  85,  86,  87,  88,  89,  90,  91,  92,  93,  94,  95,
         96,  97,  98,  99, 100, 101, 102, 103, 104, 105, 106, 107, 108,
        109, 110, 111, 112, 113, 114, 115, 116], dtype=int64),)

In [30]:
thresholds[53]

0.5840757082074639

In [31]:
precision_score(y_test, y_pred = model_pipeline.predict_proba(X_test)[:, 1] > thresholds[53])

0.9523809523809523

In [32]:
recall_score(y_test, y_pred = model_pipeline.predict_proba(X_test)[:, 1] > thresholds[53])

0.7792207792207793

In [33]:
pd.DataFrame(
    confusion_matrix(y_test, model_pipeline.predict_proba(X_test)[:, 1] > thresholds[36]),
    index=[["actual", "actual"], ["negative", "positive"]],
    columns=[["predicted", "predicted"], ["negative", "positive"]],
)

predicted         
                 negative positive
actual negative        35        9
       positive         5       72

Мы нашли 41 негативный комментарий из 44! thresholds[8] - Найдем все позитивные комментарии. thresholds[78] - все негативные.
При thresholds[36] наибольшая точность в 88%

In [ ]:
# Здесь у меня ошибка так как не знаю откуда берется model_B поэтому я просто закомментировал
# Байес
# model_B.fit(features, y_train)
model_pipeline_B = Pipeline([
    ("vectorizer", TfidfVectorizer(tokenizer = lambda x: tokenize_sentence(x, remove_stop_words=True), token_pattern=None)),
    ("model", MultinomialNB())])

In [36]:
model_pipeline_B.fit(X_train, y_train)

Pipeline(steps=[('vectorizer',
                 TfidfVectorizer(token_pattern=None,
                                 tokenizer=<function <lambda> at 0x000001D21DBA7240>)),
                ('model', MultinomialNB())])

In [37]:
y_pred_B = model_pipeline_B.predict(X_test)
pd.DataFrame(
    confusion_matrix(y_test, y_pred_B),
    index=[["actual", "actual"], ["negative", "positive"]],
    columns=[["predicted", "predicted"], ["negative", "positive"]],
)

predicted         
                 negative positive
actual negative        33       11
       positive         8       69

In [38]:
# Оценка качества модели
accuracy = accuracy_score(y_test, y_pred_B)
report = classification_report(y_test, y_pred_B)

print(f'Accuracy: {accuracy}')
print('Classification Report:')
print(report)

Accuracy: 0.8429752066115702
Classification Report:
              precision    recall  f1-score   support

           0       0.80      0.75      0.78        44
           1       0.86      0.90      0.88        77

    accuracy                           0.84       121
   macro avg       0.83      0.82      0.83       121
weighted avg       0.84      0.84      0.84       121

